# Handlers

> Response builder functions for card stack operations (Tier 1 API).

These functions take current state + items, mutate state in place, and return
HTMX response elements. Consumers call them from their own route handlers.

In [ ]:
#| default_exp routes.handlers

In [ ]:
#| export
from typing import Any, Callable, List, Optional, Tuple

from cjm_fasthtml_card_stack.core.config import CardStackConfig
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.models import CardStackState, CardStackUrls
from cjm_fasthtml_card_stack.components.viewport import render_all_slots_oob, render_viewport
from cjm_fasthtml_card_stack.components.progress import render_progress_indicator
from cjm_fasthtml_card_stack.helpers.focus import render_focus_oob

## Response Builders

Composable functions that build HTMX response parts. Consumers can use these
directly or call the higher-level `card_stack_navigate` functions.

In [ ]:
#| export
def build_slots_response(
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
) -> List[Any]:  # OOB slot elements (3 viewport sections)
    """Build OOB slot updates for the viewport sections only."""
    return render_all_slots_oob(
        card_items=card_items,
        state=state,
        config=config,
        ids=ids,
        urls=urls,
        render_card=render_card,
    )

In [ ]:
#| export
def build_nav_response(
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
    progress_label: str = "Item",  # Label for progress indicator
) -> Tuple:  # OOB elements (slots + progress + focus)
    """Build full OOB response for navigation: slots + progress + focus inputs."""
    slots_oob = build_slots_response(
        card_items=card_items, state=state, config=config,
        ids=ids, urls=urls, render_card=render_card,
    )
    progress_oob = render_progress_indicator(
        state.focused_index, len(card_items), ids,
        label=progress_label, oob=True,
    )
    focus_oob = render_focus_oob(state.focused_index, ids)
    return (*slots_oob, progress_oob, *focus_oob)

## Navigation

In [ ]:
#| export
def card_stack_navigate(
    direction: str,  # "up", "down", "first", "last", "page_up", "page_down"
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state (mutated in place)
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
    progress_label: str = "Item",  # Label for progress indicator
) -> Tuple:  # OOB elements (slots + progress + focus)
    """Navigate to a different item. Mutates state.focused_index in place."""
    total = len(card_items)
    if total == 0:
        return build_slots_response(
            card_items, state, config, ids, urls, render_card
        )

    # Page jump = visible_count - 1 (one overlap card), minimum 1
    page_jump = max(1, state.visible_count - 1)

    direction_map = {
        "up": max(0, state.focused_index - 1),
        "down": min(total - 1, state.focused_index + 1),
        "first": 0,
        "last": total - 1,
        "page_up": max(0, state.focused_index - page_jump),
        "page_down": min(total - 1, state.focused_index + page_jump),
    }
    state.focused_index = direction_map.get(direction, state.focused_index)

    return build_nav_response(
        card_items, state, config, ids, urls, render_card,
        progress_label=progress_label,
    )

In [ ]:
#| export
def card_stack_navigate_to_index(
    target_index: int,  # Target item index to navigate to
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state (mutated in place)
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
    progress_label: str = "Item",  # Label for progress indicator
) -> Tuple:  # OOB elements (slots + progress + focus)
    """Navigate to a specific item index. Mutates state.focused_index in place."""
    total = len(card_items)
    if total == 0:
        return build_slots_response(
            card_items, state, config, ids, urls, render_card
        )

    state.focused_index = max(0, min(total - 1, target_index))

    return build_nav_response(
        card_items, state, config, ids, urls, render_card,
        progress_label=progress_label,
    )

## Viewport Update

In [ ]:
#| export
def card_stack_update_viewport(
    visible_count: int,  # New number of visible cards
    card_items: List[Any],  # All data items
    state: CardStackState,  # Current card stack state (mutated in place)
    config: CardStackConfig,  # Card stack configuration
    ids: CardStackHtmlIds,  # HTML IDs for this instance
    urls: CardStackUrls,  # URL bundle for navigation
    render_card: Callable,  # Card renderer callback
) -> Tuple:  # OOB section elements (3 viewport sections)
    """Update viewport with new card count via OOB section swaps. Mutates state.visible_count in place."""
    state.visible_count = visible_count
    return tuple(build_slots_response(
        card_items=card_items,
        state=state,
        config=config,
        ids=ids,
        urls=urls,
        render_card=render_card,
    ))

## Preference Persistence

In [ ]:
#| export
def card_stack_save_width(
    state: CardStackState,  # Current card stack state (mutated in place)
    card_width: int,  # Card stack width in rem
    config: CardStackConfig,  # Card stack configuration (for clamping bounds)
) -> None:  # No response (swap=none on client)
    """Save card stack width. Mutates state.card_width in place."""
    state.card_width = max(config.card_width_min, min(config.card_width_max, card_width))

In [ ]:
#| export
def card_stack_save_scale(
    state: CardStackState,  # Current card stack state (mutated in place)
    card_scale: int,  # Card stack scale percentage
    config: CardStackConfig,  # Card stack configuration (for clamping bounds)
) -> None:  # No response (swap=none on client)
    """Save card stack scale. Mutates state.card_scale in place."""
    state.card_scale = max(config.card_scale_min, min(config.card_scale_max, card_scale))

## Tests

In [ ]:
from cjm_fasthtml_card_stack.core.config import _reset_prefix_counter
from cjm_fasthtml_card_stack.core.models import CardStackState, CardRenderContext, CardStackUrls
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds

# Simple render_card for testing
from fasthtml.common import Div, Span

def _test_render_card(item, context: CardRenderContext):
    return Div(Span(f"Item {context.index}: {item}"), cls=f"card-{context.card_role}")

# Setup test fixtures
_reset_prefix_counter()
_test_config = CardStackConfig(prefix="test")
_test_ids = CardStackHtmlIds(prefix="test")
_test_urls = CardStackUrls(
    nav_up="/nav_up", nav_down="/nav_down",
    nav_first="/nav_first", nav_last="/nav_last",
    nav_page_up="/nav_page_up", nav_page_down="/nav_page_down",
    nav_to_index="/nav_to_index",
    update_viewport="/update_viewport",
    save_width="/save_width", save_scale="/save_scale",
)
_test_items = [f"Item {i}" for i in range(20)]

print("Test fixtures ready.")

Test fixtures ready.


In [ ]:
# Test card_stack_navigate — basic down navigation
state = CardStackState(focused_index=5, visible_count=3)
result = card_stack_navigate(
    "down", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.focused_index == 6  # Mutated in place
assert len(result) > 0  # Returns OOB elements
print("Navigate down test passed!")

Navigate down test passed!


In [ ]:
# Test card_stack_navigate — boundary clamping
state = CardStackState(focused_index=0, visible_count=3)
card_stack_navigate("up", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 0  # Can't go below 0

state = CardStackState(focused_index=19, visible_count=3)
card_stack_navigate("down", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 19  # Can't go above total-1
print("Navigate boundary clamping tests passed!")

Navigate boundary clamping tests passed!


In [ ]:
# Test card_stack_navigate — first/last
state = CardStackState(focused_index=10, visible_count=3)
card_stack_navigate("first", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 0

card_stack_navigate("last", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 19
print("Navigate first/last tests passed!")

Navigate first/last tests passed!


In [ ]:
# Test card_stack_navigate — page jump uses visible_count - 1
state = CardStackState(focused_index=10, visible_count=5)
card_stack_navigate("page_down", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 14  # 10 + (5-1) = 14

state = CardStackState(focused_index=10, visible_count=3)
card_stack_navigate("page_up", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 8  # 10 - (3-1) = 8

# Single card: page jump = 1
state = CardStackState(focused_index=10, visible_count=1)
card_stack_navigate("page_down", _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card)
assert state.focused_index == 11  # 10 + max(1, 1-1) = 10 + 1 = 11
print("Page jump tests passed!")

Page jump tests passed!


In [ ]:
# Test card_stack_navigate — empty items
state = CardStackState(focused_index=0, visible_count=3)
result = card_stack_navigate(
    "down", [], state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.focused_index == 0  # Unchanged
assert len(result) > 0  # Still returns elements (empty viewport sections)
print("Empty items navigate test passed!")

Empty items navigate test passed!


In [ ]:
# Test card_stack_navigate_to_index
state = CardStackState(focused_index=0, visible_count=3)
result = card_stack_navigate_to_index(
    15, _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.focused_index == 15

# Clamps out-of-range
card_stack_navigate_to_index(
    100, _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.focused_index == 19  # Clamped to total-1

card_stack_navigate_to_index(
    -5, _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.focused_index == 0  # Clamped to 0
print("Navigate to index tests passed!")

Navigate to index tests passed!


In [ ]:
# Test card_stack_update_viewport returns OOB section tuple
from fasthtml.common import to_xml

state = CardStackState(focused_index=5, visible_count=3)
result = card_stack_update_viewport(
    7, _test_items, state, _test_config, _test_ids, _test_urls, _test_render_card
)
assert state.visible_count == 7  # Mutated in place
assert isinstance(result, tuple)  # Returns tuple of OOB sections
assert len(result) == 3  # 3 viewport sections (before, focused, after)

# Verify each section has OOB swap attribute
for section in result:
    html = to_xml(section)
    assert 'hx-swap-oob="innerHTML"' in html

# Verify section IDs
assert f'id="{_test_ids.viewport_section_before}"' in to_xml(result[0])
assert f'id="{_test_ids.viewport_section_focused}"' in to_xml(result[1])
assert f'id="{_test_ids.viewport_section_after}"' in to_xml(result[2])
print("Update viewport OOB test passed!")

Update viewport OOB test passed!


In [ ]:
# Test card_stack_save_width
state = CardStackState(card_width=80)
card_stack_save_width(state, 60, _test_config)
assert state.card_width == 60

# Clamping above max
card_stack_save_width(state, 200, _test_config)
assert state.card_width == _test_config.card_width_max  # 120

# Clamping below min
card_stack_save_width(state, 10, _test_config)
assert state.card_width == _test_config.card_width_min  # 30
print("Save width tests passed!")

Save width tests passed!


In [ ]:
# Test card_stack_save_scale
state = CardStackState(card_scale=100)
card_stack_save_scale(state, 150, _test_config)
assert state.card_scale == 150

# Clamping above max
card_stack_save_scale(state, 300, _test_config)
assert state.card_scale == _test_config.card_scale_max  # 200

# Clamping below min
card_stack_save_scale(state, 10, _test_config)
assert state.card_scale == _test_config.card_scale_min  # 50
print("Save scale tests passed!")

Save scale tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()